In [1]:
import pandas as pd

# Exploring

In [2]:
df_energy = pd.read_csv("Data/EnergyProduction.csv", header = 0)

In [3]:
def convert_float(string): #problème aperçu quand en train de filtrer pour les renouvelables - il y a d'ans où on a aucune information
    if string == "Not Available":
        return(0)
    else:
        return(float(string))

In [4]:
df_energy['Year'] = df_energy['YYYYMM'].apply(lambda x: int(str(x)[:4]))
df_energy['Month'] = df_energy['YYYYMM'].apply(lambda x: int(str(x)[4:6]))
df_energy['Value'] = df_energy['Value'].apply(lambda x: convert_float(x))

In [5]:
df_energy.sample(4) #il y a des valeurs négatifs pour value parfois

,MSN,YYYYMM,Value,Column_Order,Description,Unit,Year,Month
3422,HPETPUS,198212,0.000,6,Electricity Net Generation From Hydroelectric ...,Million Kilowatthours,1982,12
7911,ELETPUS,197601,178608.612,13,Electricity Net Generation Total (including fr...,Million Kilowatthours,1976,1
1749,NGETPUS,200502,44912.617,3,"Electricity Net Generation From Natural Gas, A...",Million Kilowatthours,2005,2
4237,HVETPUS,199504,24574.056,7,Electricity Net Generation From Conventional H...,Million Kilowatthours,1995,4


In [6]:
print('Types of energy and Column_Order index \n')
for energy in df_energy['Description'].unique():
        name = energy.split(',')[0][31:]
        number = df_energy.where(df_energy['Description']==energy)['Column_Order'].unique()[1]
        print(f'{name} = {number}')

Types of energy and Column_Order index 

 Coal = nan
 Petroleum = 2.0
 Natural Gas = 3.0
 Other Gases = 4.0
 Nuclear Electric Power = 5.0
 Hydroelectric Pumped Storage = 6.0
 Conventional Hydroelectric Power = 7.0
 Wood = 8.0
 Waste = 9.0
 Geothermal = 10.0
 Solar = 11.0
 Wind = 12.0
l (including from sources not shown) = 13.0


# Data treatment Coal, Wind, Solar and Natural Gas

In [7]:
energy_index = {'coal':1,'gas':3, 'solar':11, 'wind':12}

def regression(annual_value,year):
    return(annual_value/12)

def df_to_annual(df):
    df = df.where(df['Month']==13).dropna()
    df.drop(['YYYYMM'], axis = 1, inplace = True)
    return(df)

def df_to_month(df):
    for year in range(1949,1973,1):
        for month in range(1,13,1):
            i = (year - 1949)*12 + month - 1
            if month<10:
                yyyymm = int(str(year) + '0' + str(month))
            else:
                yyyymm = int(str(year) + str(month))
            value = df.loc[df['Year']==year]['Value'].values[0]
            new_row = pd.DataFrame({'YYYYMM': yyyymm, 'Year': year, 'Month': month, 'Value': regression(value,year)}, index = [i])
            df = df.append(new_row, ignore_index = False)
    return(df)

for energy in energy_index:
    df = df_energy.where(df_energy['Column_Order'] == energy_index[energy]).dropna()
    df.drop(['Column_Order','Description','MSN'], axis = 1, inplace = True)
    df_annual = df_to_annual(df)
    df_month = df_to_month(df)
    file_annual = 'treated_data/' + energy + '_treated.csv'
    file_month = 'treated_data/' + energy + 'monthly_treated.csv'
    df_annual.to_csv(file_annual, sep=';', index=True)
    df_month.to_csv(file_month, sep=';', index=True)

# Renewable production

In [8]:
df_renewable = df_energy.where((df_energy['Column_Order']>=8) &(df_energy['Column_Order']<=12)).dropna()
df_renewable['Year'] = df_renewable['YYYYMM'].apply(lambda x: int(str(x)[:4]))
df_renewable['Month'] = df_renewable['YYYYMM'].apply(lambda x: int(str(x)[4:6]))
df_renewable['Value'] = df_energy['Value'].apply(lambda x: convert_float(x))
df_renewable.drop(['Column_Order','Description','Unit','MSN'], axis = 1, inplace = True)

In [9]:
df_renewable_annual_ = df_renewable.where(df_renewable['Month']==13).dropna()
df_renewable_annual_.drop(['Month'], axis = 1, inplace = True)

In [10]:
df_renewable_annual = df_renewable_annual_.groupby('Year').sum()

In [11]:
df_renewable_month__ = df_renewable.where(df_renewable['Month']!=13).dropna()
df_tool = df_renewable_month__.groupby(['Year','Month']).sum()
df_renewable_month = pd.DataFrame()

def regression(annual_value,year):
    return(annual_value/12)

#Construction du DataFrame
for year in range(1949,1973,1):
    for month in range(1,13,1):
        index = (year - 1949)*12 + month - 1
        if month<10:
            yyyymm = int(str(year) + '0' + str(month))
        else:
            yyyymm = int(str(year) + str(month))
        value = df_renewable.loc[df_renewable['Year']==year]['Value'].values[0]
        new_row = pd.DataFrame({'YYYYMM': yyyymm, 'Year': year, 'Month': month, 'Value': regression(value,year)}, index=[index])
        #print(new_row)
        df_renewable_month=df_renewable_month.append(new_row, ignore_index = False)
        
for i in range(len(df_tool)): # ici on utilise le range normal, 0 -> 2021-1973, puisque les ans presents dans df_renewable_month sont les indexes
        year = df_tool.iloc[i].name[0]
        month = df_tool.iloc[i].name[1]
        index = (year - 1949)*12 + month
        if month<10:
            yyyymm = int(str(year)[:-2] + '0' + str(month)[:-2]) #[:-2] pour 1973.0 -> 1973
        else:
            yyyymm = int(str(year)[:-2] + str(month)[:-2])
        value = df_tool.iloc[i]['Value']
        new_row = pd.DataFrame({'YYYYMM': yyyymm, 'Year': year, 'Month': month, 'Value': value}, index=[index])
        df_renewable_month=df_renewable_month.append(new_row, ignore_index = False)